In [24]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.cluster import KMeans
from scipy.stats import multivariate_normal

size=[]
weights=[]
cov=[]
means=[]

def gauss(X, mean_vector, covariance_matrix):
    if (np.abs(np.linalg.det(covariance_matrix))==0):
        print("ERROR")
     # a= (2*np.pi)**(-len(X)/2)*np.abs(np.prod((np.linalg.eigvals(covariance_matrix))))**(-1/2)*np.exp(-np.dot(np.dot((X-mean_vector).T, np.linalg.pinv(covariance_matrix)), (X-mean_vector))/2)
    b= (2*np.pi)**(-len(X)/2)*(np.linalg.det(covariance_matrix))**(-1/2)*np.exp(-np.dot(np.dot((X-mean_vector).T, np.linalg.inv(covariance_matrix)), (X-mean_vector))/2)
     # c= ((1/(((2*math.pi)**(X.shape[0]/2))*((np.linalg.det(covariance_matrix))**0.5)))*math.exp(-0.5*np.matmul(np.matmul((X-mean_vector).T,np.linalg.pinv(covariance_matrix)),(X-mean_vector))))
     # return (2*np.pi)**(-len(X)/2)*np.linalg.det(covariance_matrix)**(-1/2)*np.exp(-np.dot(np.dot((X-mean_vector).T, np.linalg.inv(covariance_matrix)), (X-mean_vector))/2)

    return b

# The only hyperparameter is k ( no.of components for each class)
k=3 
train=['coast','forest','opencountry','street','tallbuilding']

for c, train_file in enumerate(train):
    data=pd.read_csv('dataset/'+train_file+'/train.csv')
    data=data.to_numpy()
    X=data[:,1:]
    size.append(len(X))
    print(f"\n\n\nClass {c}")
#     print(size)
    kmeans=KMeans(n_clusters=k,random_state=0).fit(X)
    # kmeans=KMeans(n_clusters=k).fit(X)
    means_old=kmeans.cluster_centers_
    labels=kmeans.labels_


    N=len(X)
    r_old=np.zeros((len(X),k)) # form a Z ( indicator ) matrix

    for i in range(len(X)):
        r_old[i,labels[i]]=1

    Nq_old=np.sum(r_old,axis=0) # sum conatins the number of elements belonging
                                 # to each cluster

    print("\nOriginal effective number of elements in each cluster")
    print(Nq_old)
    # Initialization

    #cov2 is a 3-d array containing the covariance matrix of each cluster
    cov_old=np.zeros([k,X.shape[1],X.shape[1]])
    Wq_old =np.zeros([k,1]) ## weight of each cluster

    for i in range(k):
        Nq=Nq_old[i]
        Wq_old[i]= Nq/N
        tp=np.zeros([X.shape[1],X.shape[1]])

        for p in range(X.shape[0]):
            le=X[p,:]-means_old[i]
            le=np.reshape(le,[le.shape[0],1])
            tp=tp+r_old[p,i]*(np.dot(le,le.T))
        tp=tp/Nq

#         d= np.diag(tp)
#         tp=np.diag(d)
        cov_old[i,:,:]=tp.copy()

    ll_old= 0.0
    for n in range(len(X)):
        ll_old = ll_old + np.log(sum([Wq_old[j]*multivariate_normal.pdf(X[n], means_old[j], cov_old[j],allow_singular=True) for j in range(k)]))

    print(f"\nInitial log-likehood = {ll_old}")

    convergence=False
    iter_convergence=0
    run=0
    runs=1000
    epsilon=100

    while (convergence == False and run<runs):

        # ''' --------------------------   E - STEP   -------------------------- '''

        # Initiating the r matrix, every row contains the probabilities
        # for every cluster for this row

        r_new = np.zeros((len(X), k))  # responsibilty matrix

        # Calculating the r matrix
        for n in range(len(X)):
            for i in range(k):
                r_new[n][i] = Wq_old[i] * multivariate_normal.pdf(X[n], means_old[i], cov_old[i],allow_singular=True)
                r_new[n][i] /= sum([Wq_old[j]*multivariate_normal.pdf(X[n], means_old[j], cov_old[j],allow_singular=True) for j in range(k)])

        # Calculating the N effective elemts fro each component
        Nq_new = np.sum(r_new, axis=0)


        # ''' --------------------------   M - STEP   -------------------------- '''


        # Updating the weights list
        Wq_new =np.zeros([k,1]) ## weight of each cluster
        for i in range(k):
            Wq_new[i]= Nq_new[i]/ N


        # Initializing the mean vector as a zero vector
        means_new = np.zeros((k, len(X[0])))

        # Updating the mean vector
        for i in range(k):
            for n in range(len(X)):
                means_new[i] = means_new[i] + r_new[n][i] * X[n]
            means_new[i] = means_new [i]/Nq_new[i]



        # Initiating the list of the covariance matrixes
        cov_new =np.zeros([k,X.shape[1],X.shape[1]])

        # Updating the covariance matrices
        for i in range(k):
            Nq=Nq_new[i]
            tp=np.zeros([X.shape[1],X.shape[1]])

            for p in range(X.shape[0]):
                le=X[p,:]-means_new[i]
                le=np.reshape(le,[le.shape[0],1])
                tp=tp+r_new[p,i]*(np.dot(le,le.T))

            tp=tp/Nq
#             d= np.diag(tp)
#             tp=np.diag(d)
            cov_new[i,:,:]=tp.copy()


        # print(f"\nRun= {run}\n")
#         print(np.sum(Nq_new))
#         print("\nWeights\n")
#         print(np.sum(Wq_new))
#         print(Wq_new)
#         print(np.sum(r_new))
#         print("\n------------------")

        # Calculating log-likelhood
        ll_new=0
        for n in range(len(X)):
            ll_new = ll_new + np.log(sum([Wq_new[j]*multivariate_normal.pdf(X[n], means_new[j], cov_new[j],allow_singular=True) for j in range(k)]))

    #     print(ll_new)
        diff=ll_new-ll_old

    #     print(diff)

        #Convergence condition
        if diff < 1e-3:
            iter_convergence=run
            convergence=True
            break

        else:
            ll_old=ll_new.copy()
            Wq_old= Wq_new.copy()
            means_old=means_new.copy()
            cov_old=cov_new.copy()

        run= run +1

    if convergence==True and run!=runs:
        print("Iterations for convergence=",iter_convergence)
    else:
        print("Estimate has not converged yet, more runs needed")
    print(f"Final log-likehood = {ll_new}")

    print("\nEffective number of elements in each cluster is")
    print(Nq_new)
#     ass=np.sum(Nq_new)
#     print(ass)
    weights.append(Wq_new)
    means.append(means_new)
    cov.append(cov_new)

print("\n##############################################################################")








Class 0

Original effective number of elements in each cluster
[107.  88.  56.]

Initial log-likehood = [6292.33397387]
Iterations for convergence= 14
Final log-likehood = [6473.1081942]

Effective number of elements in each cluster is
[96.01512101 91.78637783 63.19850117]



Class 1

Original effective number of elements in each cluster
[72. 97. 60.]

Initial log-likehood = [9176.39023618]
Iterations for convergence= 26
Final log-likehood = [9418.93006826]

Effective number of elements in each cluster is
[71.0511302  75.81482239 82.13404741]



Class 2

Original effective number of elements in each cluster
[ 88.  97. 102.]

Initial log-likehood = [7314.44371518]
Iterations for convergence= 25
Final log-likehood = [7525.9882791]

Effective number of elements in each cluster is
[ 96.20345373 100.45246845  90.34407782]



Class 3

Original effective number of elements in each cluster
[75. 86. 43.]

Initial log-likehood = [7853.03451778]
Iterations for convergence= 6
Final log-likehood

In [44]:
size=np.array(size)
prior_class=size/np.sum(size)

validation_set=['coast','forest','opencountry','street','tallbuilding']
# validation_set=['train_1.csv','train_2.csv','train_3.csv','train_4.csv','train_5.csv']


valid_data=pd.DataFrame()
test_data=pd.DataFrame()

for ind, valid_file in enumerate(validation_set):

    X_valid=pd.read_csv('dataset/'+valid_file+'/dev.csv')
    X_valid['y']=ind
    msk = np.random.rand(len(X_valid)) < 0.5  #50-50 splits
    #print(X_valid[msk])
    valid_data=pd.concat([valid_data,X_valid[msk]],ignore_index=True)
    test_data=pd.concat([test_data,X_valid[~msk]],ignore_index=True)
    
#print(valid_data)
#print(test_data)  
 
#print(valid_data)
#print(test_data)

    

                    image_names         0         1         2         3  \
0             coast_natu777.jpg  0.256210  0.246597  0.150299  0.097305   
1             coast_natu817.jpg  0.197052  0.157166  0.153442  0.105743   
2             coast_natu642.jpg  0.021164  0.301926  0.255981  0.036484   
3             coast_natu619.jpg  0.002441  0.135590  0.419647  0.273682   
4             coast_natu804.jpg  0.000015  0.024017  0.022324  0.103210   
..                          ...       ...       ...       ...       ...   
156     tallbuilding_urb492.jpg  0.106720  0.388992  0.157837  0.118271   
157     tallbuilding_urb974.jpg  0.475754  0.189987  0.072449  0.080658   
158  tallbuilding_urban1158.jpg  0.162399  0.419479  0.139389  0.079437   
159     tallbuilding_urb390.jpg  0.178162  0.076935  0.165024  0.132324   
160      tallbuilding_sky21.jpg  0.241379  0.315735  0.075287  0.031540   

            4         5         6         7         8  ...        15  \
0    0.074341  0.069366  0.

In [40]:
"""
for ind, valid_file in enumerate(validation_set):

    
    X_valid=X_valid.to_numpy()
    X_valid=X_valid[:,1:]

    index=[]

    for n in range(len(X_valid)):
        ll_n=[]
        for i in range(len(validation_set)):
            ll= np.log(sum([weights[i][j]*multivariate_normal.pdf(X_valid[n], means[i][j], cov[i][j],allow_singular=True)  for j in range(k)])) + np.log(prior_class[i])
            ll_n.append(ll)
        ll_n=np.array(ll_n)
        index.append(np.argmax(ll_n))

    p=index.count(ind)
    prob=p/len(index)

    print(prob)
    
    """
    

'\nfor ind, valid_file in enumerate(validation_set):\n\n    \n    X_valid=X_valid.to_numpy()\n    X_valid=X_valid[:,1:]\n\n    index=[]\n\n    for n in range(len(X_valid)):\n        ll_n=[]\n        for i in range(len(validation_set)):\n            ll= np.log(sum([weights[i][j]*multivariate_normal.pdf(X_valid[n], means[i][j], cov[i][j],allow_singular=True)  for j in range(k)])) + np.log(prior_class[i])\n            ll_n.append(ll)\n        ll_n=np.array(ll_n)\n        index.append(np.argmax(ll_n))\n\n    p=index.count(ind)\n    prob=p/len(index)\n\n    print(prob)\n    \n    '

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

predicted=[]
real=[]
k=3
#print(len(validation_set))
for i in range(len(valid_data)):

    X_valid=(valid_data.loc[i,:]).to_numpy()
    #print(X_valid)
    X_valid=X_valid[1:]
    Y_valid=X_valid[-1]
    X_valid=X_valid[:-1]
    real.append(Y_valid)
    

    #for n in range(len(X_valid)):
    ll_n=[]
    for i in range(len(validation_set)):
        ll= np.log(sum([weights[i][j]*multivariate_normal.pdf(X_valid, means[i][j], cov[i][j],allow_singular=True)  for j in range(k)])) + np.log(prior_class[i])
        ll_n.append(ll)
    ll_n=np.array(ll_n)
    predicted.append(np.argmax(ll_n))
    
    #p=index.count(ind)
    #prob=p/len(index)

    #print(prob)
    
print("accuracy on validation set using full covariance matrix and k=3 is "+str(accuracy_score(real,predicted)*100))
    

<ipython-input-41-e0e369bfe355>:21: RuntimeWarning: divide by zero encountered in log
  ll= np.log(sum([weights[i][j]*multivariate_normal.pdf(X_valid, means[i][j], cov[i][j],allow_singular=True)  for j in range(k)])) + np.log(prior_class[i])


accuracy on validation set using full covariance matrix and k=3 is 65.71428571428571
